This script performs analyses to check how many mice pass the currenty set criterion for ephys.

In [2]:
import datajoint as dj
dj.config['database.host'] = 'datajoint.internationalbrainlab.org'

from ibl_pipeline import subject, acquisition, action, behavior, reference, data
from ibl_pipeline.analyses.behavior import PsychResults, SessionTrainingStatus
from ibl_pipeline.utils import psychofit as psy
from ibl_pipeline.analyses import behavior as behavior_analysis

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Connecting lacerbi@datajoint.internationalbrainlab.org:3306


In [56]:
sess_ephys = (acquisition.Session * subject.Subject * behavior_analysis.SessionTrainingStatus ) & 'task_protocol LIKE "%ephys%"'
#               & 'task_protocol LIKE "%biased%"' & 'session_start_time < "2019-09-30"')

first_pass_map_repeated = [('ZM_2241','2020-01-30'),('CSHL049','2020-01-08'),('CSHL047','2020-01-20'),('CSHL051','2020-02-05'),
                  ('KS023','2019-12-10'),('KS020','2020-02-07'),('CSHL054','2020-02-12'),('SWC_023','2020-02-13'),
                  ('NYU-12','2020-01-22'),('ibl_witten_13','2019-12-03'),('ibl_witten_14','2019-12-11'),('SWC_014','2019-12-10'),
                  ('CSHL052','2020-02-21'),('CSHL045','2020-02-25'),('DY_009','2020-02-27'),('NYU-11','2020-02-20'),
                  ('DY_008','2020-03-03'),('CSHL059','2020-03-06'),('DY_013','2020-03-07')]

df = pd.DataFrame(sess_ephys)

In [113]:
session_dates = df['session_start_time'].apply(lambda x : x.strftime("%Y-%m-%d"))

# First, count all mice

total = len(df.index)
good_enough = np.sum(df['good_enough_for_brainwide_map'])
prc = good_enough / total * 100
print('Total # of ephys sessions: '+ str(total))
print('Total # of sessions good_enough_for_brainwide_map: ' + str(good_enough) + ' (' + "{:.1f}".format(prc) + ' %)')

# Now, consider only mice in the first pass map, repeated site

count = 0
for (mouse_name,session_date) in first_pass_map:
    tmp = df[(df['subject_nickname']  == mouse_name) & (session_dates == session_date)]
    count = count + np.sum(tmp['good_enough_for_brainwide_map'])
    
total = len(first_pass_map_repeated)
good_enough = count
prc = good_enough / total * 100
print('Total # of ephys sessions in first pass map, repeated site: '+ str(total))
print('Total # of sessions good_enough_for_brainwide_map in first pass map, repeated site: ' + str(good_enough) + ' (' + "{:.1f}".format(prc) + ' %)')


Total # of ephys sessions: 200
Total # of sessions good_enough_for_brainwide_map: 114 (57.0 %)
Total # of ephys sessions in first pass map, repeated site: 19
Total # of sessions good_enough_for_brainwide_map in first pass map, repeated site: 10 (52.6 %)
